# 演習9.2

警告を非表示に

In [1]:
import warnings
warnings.filterwarnings('ignore')

IMDBデータは映画のレビューに対して、P/N(肯定/否定)のラベルが付いた学習データです。学習用に25000事例、評価用に25000事例用意されていて、PNの割合はそれぞれ50%です。
各レビューは単語列ではなく、単語インデックスの系列として表現されています。

ここでは、頻度上位10000語を対象とし、データの大きさは先頭の20単語に限定します。

In [2]:
from keras.datasets import imdb
from keras import preprocessing
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
max_features = 10000
maxlen = 20
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
X_train = preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

Using TensorFlow backend.


単語インデックスを単語に戻して、元のデータを確認します。インデックスは"padding", "start of sequence","unknown"にそれぞれ0,1,2が割り当てられているので、3つずらして対応させます。

In [3]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_train[0]])
decoded_review

"story was so lovely because it was true and was someone's life after all that was shared with us all"

単純なRNNを構成して学習させます。

In [4]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=10, batch_size=128,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s 69us/step - loss: 0.6073 - acc: 0.6509 - val_loss: 0.5779 - val_acc: 0.6936
Epoch 2/10
20000/20000 [==============================] - 1s 61us/step - loss: 0.4414 - acc: 0.7965 - val_loss: 0.5283 - val_acc: 0.7338
Epoch 3/10
20000/20000 [==============================] - 1s 61us/step - loss: 0.3491 - acc: 0.8484 - val_loss: 0.5590 - val_acc: 0.7108
Epoch 4/10
20000/20000 [==============================] - 1s 62us/step - loss: 0.2602 - acc: 0.8959 - val_loss: 0.6101 - val_acc: 0.7296
Epoch 5/10
20000/20000 [==============================] - 1s 63us/step - loss: 0.1784 - acc: 0.9312 - val_loss: 0.7043 - val_acc: 0.7154
Epoch 6/10
20000/20000 [==============================] - 1s 59us/step - loss: 0.1169 - acc: 0.9582 - val_loss: 0.7705 - val_acc: 0.7110
Epoch 7/10
20000/20000 [==============================] - 1s 59us/step - loss: 0.0683 - acc: 0.9781 - val_loss: 0.9151 - val_acc:

In [5]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.3222395536994933
Test accuracy: 0.69772


RNNユニットをLSTMに変更して性能の変化を確認します。

In [6]:
from keras.layers import LSTM
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=10, batch_size=128,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 3s 140us/step - loss: 0.5843 - acc: 0.6873 - val_loss: 0.5195 - val_acc: 0.7432
Epoch 2/10
20000/20000 [==============================] - 2s 106us/step - loss: 0.4436 - acc: 0.7943 - val_loss: 0.4994 - val_acc: 0.7566
Epoch 3/10
20000/20000 [==============================] - 2s 107us/step - loss: 0.3959 - acc: 0.8226 - val_loss: 0.4878 - val_acc: 0.7548
Epoch 4/10
20000/20000 [==============================] - 2s 107us/step - loss: 0.3694 - acc: 0.8361 - val_loss: 0.5104 - val_acc: 0.7478
Epoch 5/10
20000/20000 [==============================] - 2s 109us/step - loss: 0.3506 - acc: 0.8476 - val_loss: 0.5043 - val_acc: 0.7484
Epoch 6/10
20000/20000 [==============================] - 2s 106us/step - loss: 0.3355 - acc: 0.8538 - val_loss: 0.5490 - val_acc: 0.7504
Epoch 7/10
20000/20000 [==============================] - 2s 123us/step - loss: 0.3225 - acc: 0.8620 - val_loss: 0.5150 - v

In [7]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.590690350780487
Test accuracy: 0.74824
